# TABLE OF CONTENT : 
 1. [Scrapping Wikipedia Web page](#1)<br>
 2. [Adding coordinates of neighborhoods](#2)<br>

In [1]:
#Installing beautifulsoup package
#!pip install beautifulsoup4

In [2]:
#import of libraries necessary to scrap the web page 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

# FIRST PART - SCRAPPING WIKIPEDIA WEB PAGE <a id="1"></a>

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#parsing the web page wikipedia
page_response=requests.get(url,timeout=5)
page_content = BeautifulSoup(page_response.content, 'lxml')

In [5]:
#creation of the data frame containing the table from wikipedia page : 
# varibale with all the tables found in the page 
tables=page_content.find_all('table')
#focus on the table we are interested in : 
table=tables[0].tbody
#creation of our data frame :
df_table=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
#iteration in the table in order to extract content and add it to our dataframe 
for row in table.find_all('tr') : 
    cols=row.find_all('td')
    try : 
        df_table=df_table.append({'PostalCode':cols[0].text,
                                          'Borough': cols[1].text, 
                                          'Neighborhood': cols[2].text, 
                                         }, ignore_index=True)
    except: 
        pass

df_table

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
...,...,...,...
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,Mimico NW / The Queensway West / South of Bloo...


In [6]:
#data frame cleaning by removin the '\n' using  REGEX: 
import re
for index,row in df_table.iterrows():
    for col in df_table.columns:
        row[col]= re.sub(r'\n','',row[col])
df_table

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [7]:
#drop rows where no borough were attributed and copy borough neighborhood where no neighborood
for index, row in df_table.iterrows():
    if row['Borough']=='Not assigned':
        df_table.drop([index],inplace=True)
    if row['Neighborhood']== '':
        row['Neighborhood']=row['Borough']
df_table.reset_index(inplace=True,drop=True)
df_table.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
# checking if any null values are stil in the data frame : 
df_clean=df_table.isnull()
for column in df_clean.columns: 
    print(column)
    print(df_clean[column].value_counts())

PostalCode
False    103
Name: PostalCode, dtype: int64
Borough
False    103
Name: Borough, dtype: int64
Neighborhood
False    103
Name: Neighborhood, dtype: int64


In [9]:
df_table.shape

(103, 3)

In [10]:
import csv
df_table.to_csv('postal_code.csv')

# Adding coordinates of neighborhoods <a id="2"></a>


In [11]:
# instal of geocoder 
#!pip install geocoder

In [12]:
import geocoder # import geocoder

In [ ]:
import geopy
geopy.geocoders.options.default_timeout = 7
# initialize your variable to None
location = None
geolocator = Nominatim(user_agent="TomTom")
for index,code in enumerate(df_table['PostalCode']): 
# loop until you get the coordinates 
    adress='Toronto, Ontario,{},Canada'.format(code)
    location =  geolocator.geocode(adress)
    if location != None :
        latitude = location.latitude
        longitude = location.longitude
        df_table.at[index,'latitude'] = latitude
        df_table.at[index,'longitude'] = longitude
    elif location ==None: 
        df_table.at[index,'latitude'] = np.nan
        df_table.at[index,'longitude'] = np.nan

In [ ]:
df_test=df_table.isnull()
df_test['latitude'].value_counts()

In [ ]:
# initialize your variable to None
lat_lng_coords = None
for index,code in enumerate(df_table['PostalCode']): 
    postal_code=code
    print(index, 'st row to complete')
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google(' Toronto, Ontario,{},Canada'.format(postal_code))
        lat_lng_coords = g.latlng
    df_table.at[index,'latitude'] = lat_lng_coords[0]
    df_table.at[index,'longitude'] = lat_lng_coords[1]
    print(index, 'st row completed')
    lat_lng_coords = None

0 st row to complete
